# SQL

DROP TABLE covid_result;
DROP TABLE covid_live;
DROP TABLE covid_vaccine_center;

CREATE TABLE covid_result(
sequence int PRIMARY KEY,
result_date DATE,
location VARCHAR(20),
increment_count INT,
total_count INT
);

CREATE TABLE covid_live(
live_date date PRIMARY KEY,
seoul INT,
incheon INT,
gwangju INT,
daejeon INT,
daegu INT,
busan INT,
ulsan INT,
sejong INT,
gyeonggi INT,
gangwon INT,
chungbuk INT,
chungnam INT,
jeonbuk INT,
jeonnam INT,
gyeongbuk INT,
gyeongnam INT,
jeju INT
);

CREATE TABLE covid_vaccine_center(
center_name VARCHAR(200),
facility_name VARCHAR(200),
phone_number VARCHAR(40),
location VARCHAR(20),
address VARCHAR(200) PRIMARY KEY,
zip_code VARCHAR(20),
lat DOUBLE,
lng DOUBLE
);

SELECT * FROM covid_result;
SELECT * FROM covid_live;
SELECT * FROM covid_vaccine_center;

COMMIT;

In [78]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
import datetime as dt

date = dt.datetime.now() # 20210514일 경우 20210513의 확진자 집계 정보가 나옴
date = date.strftime('%Y%m%d')
print(date)

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson' # 서비스 URL
apiKey = 'serviceKey=' + 'WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D' # 인증키
pageNo = 'pageNo=' + str(1) # 페이지 번호
numOfRows = 'numOfRows=' + str(30) # 한 페이지 결과 수
startCreateDt = 'startCreateDt=' + str(date) # 데이터 생성일 시작 범위
endCreateDt = 'endCreateDt=' + str(date) # 데이터 생성일 종료 범위
params = pageNo + '&' + numOfRows  + '&' + startCreateDt  + '&' + endCreateDt # 파라메터 합치는 과정
open_url = url + '?' + apiKey + '&' + params # 요청 URL 완성

print(open_url + '\n') # 요청한 링크, 링크 누르면 데이터 확인 가능

res = requests.get(open_url) # 응답받은 데이터를 res에 저장
soup = BeautifulSoup(res.content, 'html.parser') # bs에 알맞게 변환

items = soup.findAll('item') # 데이터에서 item 태그들을 찾아 ResultSet으로 반환

result = [] # item 데이터를 넣을 tuple 생성

for one in items:
    print('sequence : ' + one.find('seq').string)
    print('수정 시간 : ' + one.find('createdt').string) # 나중에는 수정 시간 하나만 출력해도 될 거 같음
    print('지역 : ' + one.find('gubun').string)
    print('일일 확진자 수 : ' + one.find('incdec').string)
    print('누적 확진자 수 : ' + one.find('defcnt').string)
    sequence = str(one.find('seq').string)
    createDt = one.find('createdt').string[:10] # ex) 2021-05-18
    splitDate = createDt.split('-')
    #실제로는 하루 전 날의 통계 데이터이기 때문에 str을 datetime로 변환 후 -1일 연산
    date = dt.date(int(splitDate[0]), int(splitDate[1]), int(splitDate[2])) - dt.timedelta(days=1)
    result_date = date.strftime('%Y-%m-%d') # 양식에 맞게 str로 변환
    location = str(one.find('gubun').string)
    increment_count = str(one.find('incdec').string)
    total_count = str(one.find('defcnt').string)
    
    data = (sequence, result_date, location, increment_count, total_count) # tuple 생성
    result.append(data) # list에 추가
    print()

# MariaDB 커넥션 생성
'''
try:
    conn = mysql.connector.connect(
        user="multi",
        password="multi",
        host="melonbottle.synology.me",
        port=3308,
        database="multi"
    )
    print(conn)
except mysql.connector.Error as e:
    print(e)
'''

cur = conn.cursor() # 커서 생성

sql = "INSERT IGNORE INTO covid_result VALUES(%s, %s, %s, %s, %s)" # 쿼리 생성

cur.executemany(sql, result) # result 만큼의 쿼리 실행


conn.commit() # 커밋

# close 작업 실행
cur.close() 
conn.close()

20210518
http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey=WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D&pageNo=1&numOfRows=10000&startCreateDt=20200305&endCreateDt=20210518

seq : 9583
수정 시간 : 2021-05-18 09:47:56.68
지역 : 검역
일일 확진자 수 : 12
누적 확진자 수 : 3690

seq : 9582
수정 시간 : 2021-05-18 09:47:56.68
지역 : 제주
일일 확진자 수 : 11
누적 확진자 수 : 897

seq : 9581
수정 시간 : 2021-05-18 09:47:56.68
지역 : 경남
일일 확진자 수 : 16
누적 확진자 수 : 4479

seq : 9580
수정 시간 : 2021-05-18 09:47:56.68
지역 : 경북
일일 확진자 수 : 23
누적 확진자 수 : 4497

seq : 9579
수정 시간 : 2021-05-18 09:47:56.679
지역 : 전남
일일 확진자 수 : 12
누적 확진자 수 : 1328

seq : 9578
수정 시간 : 2021-05-18 09:47:56.679
지역 : 전북
일일 확진자 수 : 7
누적 확진자 수 : 2120

seq : 9577
수정 시간 : 2021-05-18 09:47:56.679
지역 : 충남
일일 확진자 수 : 15
누적 확진자 수 : 3287

seq : 9576
수정 시간 : 2021-05-18 09:47:56.679
지역 : 충북
일일 확진자 수 : 13
누적 확진자 수 : 2795

seq : 9575
수정 시간 : 2021-05-18 09:47:56.679
지역 : 강원
일일 확진자 수 : 25
누적 확진자 수 : 297

seq : 9093
수정 시간 : 2021-04-25 10:37:28.099
지역 : 대구
일일 확진자 수 : 14
누적 확진자 수 : 9270

seq : 9092
수정 시간 : 2021-04-25 10:37:28.099
지역 : 부산
일일 확진자 수 : 43
누적 확진자 수 : 4889

seq : 9091
수정 시간 : 2021-04-25 10:37:28.098
지역 : 서울
일일 확진자 수 : 175
누적 확진자 수 : 36856

seq : 9090
수정 시간 : 2021-04-25 10:37:28.098
지역 : 합계
일일 확진자 수 : 644
누적 확진자 수 : 118887

seq : 9089
수정 시간 : 2021-04-24 09:43:11.137
지역 : 검역
일일 확진자 수 : 11
누적 확진자 수 : 3372

seq : 9088
수정 시간 : 2021-04-24 09:43:11.137
지역 : 제주
일일 확진자 수 : 3
누적 확진자 수 : 693

seq : 9087
수정 시간 : 2021-04-24 09:43:11.137
지역 : 경남
일일 확진자 수 : 52
누적 확진자 수 : 3683

seq : 9086
수정 시간 : 2021-04-24 09:43:11.136
지역 : 경북
일일 확진자 수 : 32
누적 확진자 수 : 3957

seq : 9085
수정 시간 : 2021-04-24 09:43:11.136
지역 : 전남
일일 확진자 수 : 1
누적 확진자 수 : 1030

seq : 9084
수정 시간 : 2021-04-24 09:43:11.136
지역 : 전북
일일 확진자 수 : 9
누적 확진자 수 : 1875

seq : 9083
수정 시간 : 2021-04-24 09:43:11.136
지역 : 충남
일일 확진자 수 : 14
누적 확진자 수 : 2940

seq : 9082
수정 시간 : 2021-04-24 09:43:11.136
지역 : 충북
일일 확진자 수 : 15
누적 확진자 수 : 2543

seq : 9081
수정 시

seq : 8732
수정 시간 : 2021-04-07 09:44:06.722
지역 : 대구
일일 확진자 수 : 15
누적 확진자 수 : 9030

seq : 8731
수정 시간 : 2021-04-07 09:44:06.722
지역 : 부산
일일 확진자 수 : 38
누적 확진자 수 : 4140

seq : 8730
수정 시간 : 2021-04-07 09:44:06.722
지역 : 서울
일일 확진자 수 : 197
누적 확진자 수 : 33196

seq : 8729
수정 시간 : 2021-04-07 09:44:06.721
지역 : 합계
일일 확진자 수 : 668
누적 확진자 수 : 106896

seq : 8728
수정 시간 : 2021-04-06 09:34:32.853
지역 : 검역
일일 확진자 수 : 5
누적 확진자 수 : 3205

seq : 8727
수정 시간 : 2021-04-06 09:34:32.853
지역 : 제주
일일 확진자 수 : 0
누적 확진자 수 : 636

seq : 8726
수정 시간 : 2021-04-06 09:34:32.853
지역 : 경남
일일 확진자 수 : 7
누적 확진자 수 : 3060

seq : 8725
수정 시간 : 2021-04-06 09:34:32.853
지역 : 경북
일일 확진자 수 : 25
누적 확진자 수 : 3619

seq : 8724
수정 시간 : 2021-04-06 09:34:32.853
지역 : 전남
일일 확진자 수 : 15
누적 확진자 수 : 937

seq : 8723
수정 시간 : 2021-04-06 09:34:32.853
지역 : 전북
일일 확진자 수 : 29
누적 확진자 수 : 1577

seq : 8722
수정 시간 : 2021-04-06 09:34:32.853
지역 : 충남
일일 확진자 수 : 6
누적 확진자 수 : 2698

seq : 8721
수정 시간 : 2021-04-06 09:34:32.852
지역 : 충북
일일 확진자 수 : 12
누적 확진자 수 : 2273

seq : 8720
수정 시간 

수정 시간 : 2021-03-17 09:35:39.389
지역 : 제주
일일 확진자 수 : 1
누적 확진자 수 : 612

seq : 8289
수정 시간 : 2021-03-17 09:35:39.389
지역 : 경남
일일 확진자 수 : 45
누적 확진자 수 : 2548

seq : 8288
수정 시간 : 2021-03-17 09:35:39.389
지역 : 경북
일일 확진자 수 : 4
누적 확진자 수 : 3381

seq : 8287
수정 시간 : 2021-03-17 09:35:39.389
지역 : 전남
일일 확진자 수 : 0
누적 확진자 수 : 905

seq : 8286
수정 시간 : 2021-03-17 09:35:39.389
지역 : 전북
일일 확진자 수 : 21
누적 확진자 수 : 1304

seq : 8285
수정 시간 : 2021-03-17 09:35:39.388
지역 : 충남
일일 확진자 수 : 6
누적 확진자 수 : 2551

seq : 8284
수정 시간 : 2021-03-17 09:35:39.388
지역 : 충북
일일 확진자 수 : 12
누적 확진자 수 : 1993

seq : 8283
수정 시간 : 2021-03-17 09:35:39.388
지역 : 강원
일일 확진자 수 : 10
누적 확진자 수 : 2056

seq : 8282
수정 시간 : 2021-03-17 09:35:39.388
지역 : 경기
일일 확진자 수 : 186
누적 확진자 수 : 26491

seq : 8281
수정 시간 : 2021-03-17 09:35:39.388
지역 : 세종
일일 확진자 수 : 3
누적 확진자 수 : 249

seq : 8280
수정 시간 : 2021-03-17 09:35:39.388
지역 : 울산
일일 확진자 수 : 4
누적 확진자 수 : 1099

seq : 8279
수정 시간 : 2021-03-17 09:35:39.387
지역 : 대전
일일 확진자 수 : 1
누적 확진자 수 : 1207

seq : 8278
수정 시간 : 2021-03-17 09:35

누적 확진자 수 : 3242

seq : 7888
수정 시간 : 2021-02-26 9:33:05.05
지역 : 전남
일일 확진자 수 : 5
누적 확진자 수 : 854

seq : 7887
수정 시간 : 2021-02-26 9:33:05.05
지역 : 전북
일일 확진자 수 : 16
누적 확진자 수 : 1164

seq : 7886
수정 시간 : 2021-02-26 9:33:05.05
지역 : 충남
일일 확진자 수 : 3
누적 확진자 수 : 2433

seq : 7885
수정 시간 : 2021-02-26 9:33:05.05
지역 : 충북
일일 확진자 수 : 10
누적 확진자 수 : 1738

seq : 7884
수정 시간 : 2021-02-26 9:33:05.05
지역 : 강원
일일 확진자 수 : 6
누적 확진자 수 : 1854

seq : 7883
수정 시간 : 2021-02-26 9:33:05.05
지역 : 경기
일일 확진자 수 : 141
누적 확진자 수 : 23243

seq : 7882
수정 시간 : 2021-02-26 9:33:05.05
지역 : 세종
일일 확진자 수 : 1
누적 확진자 수 : 219

seq : 7881
수정 시간 : 2021-02-26 9:33:05.05
지역 : 울산
일일 확진자 수 : 1
누적 확진자 수 : 1005

seq : 7880
수정 시간 : 2021-02-26 9:33:05.05
지역 : 대전
일일 확진자 수 : 6
누적 확진자 수 : 1187

seq : 7879
수정 시간 : 2021-02-26 9:33:05.05
지역 : 광주
일일 확진자 수 : 11
누적 확진자 수 : 2058

seq : 7878
수정 시간 : 2021-02-26 9:33:05.05
지역 : 인천
일일 확진자 수 : 15
누적 확진자 수 : 4391

seq : 7877
수정 시간 : 2021-02-26 09:33:05.228
지역 : 대구
일일 확진자 수 : 10
누적 확진자 수 : 8612

seq : 7876
수정 시간 : 2021-02-

...

일일 확진자 수 : 0
누적 확진자 수 : 148

seq : 2223
수정 시간 : 2020-06-08 10:35:23.23
지역 : 충북
일일 확진자 수 : 0
누적 확진자 수 : 61

seq : 2222
수정 시간 : 2020-06-08 10:35:23.23
지역 : 강원
일일 확진자 수 : 0
누적 확진자 수 : 58

seq : 2221
수정 시간 : 2020-06-08 10:35:23.23
지역 : 경기
일일 확진자 수 : 8
누적 확진자 수 : 942

seq : 2220
수정 시간 : 2020-06-08 10:35:23.23
지역 : 세종
일일 확진자 수 : 0
누적 확진자 수 : 47

seq : 2219
수정 시간 : 2020-06-08 10:35:23.23
지역 : 울산
일일 확진자 수 : 0
누적 확진자 수 : 53

seq : 2218
수정 시간 : 2020-06-08 10:35:23.23
지역 : 대전
일일 확진자 수 : 0
누적 확진자 수 : 46

seq : 2217
수정 시간 : 2020-06-08 10:35:23.23
지역 : 광주
일일 확진자 수 : 0
누적 확진자 수 : 32

seq : 2216
수정 시간 : 2020-06-08 10:35:23.23
지역 : 인천
일일 확진자 수 : 4
누적 확진자 수 : 283

seq : 2215
수정 시간 : 2020-06-08 10:35:23.23
지역 : 대구
일일 확진자 수 : 1
누적 확진자 수 : 6888

seq : 2214
수정 시간 : 2020-06-08 10:35:23.23
지역 : 부산
일일 확진자 수 : 0
누적 확진자 수 : 147

seq : 2213
수정 시간 : 2020-06-08 10:35:23.23
지역 : 서울
일일 확진자 수 : 22
누적 확진자 수 : 996

seq : 2212
수정 시간 : 2020-06-08 10:35:23.23
지역 : 합계
일일 확진자 수 : 38
누적 확진자 수 : 11814

seq : 2211
수정 시간 : 2020-

seq : 1750
수정 시간 : 2020-05-15 10:16:39.39
지역 : 전북
일일 확진자 수 : 0
누적 확진자 수 : 20

seq : 1749
수정 시간 : 2020-05-15 10:16:39.39
지역 : 충남
일일 확진자 수 : 0
누적 확진자 수 : 144

seq : 1748
수정 시간 : 2020-05-15 10:16:39.39
지역 : 충북
일일 확진자 수 : 0
누적 확진자 수 : 55

seq : 1747
수정 시간 : 2020-05-15 10:16:39.39
지역 : 강원
일일 확진자 수 : 0
누적 확진자 수 : 55

seq : 1746
수정 시간 : 2020-05-15 10:16:39.39
지역 : 경기
일일 확진자 수 : 3
누적 확진자 수 : 717

seq : 1745
수정 시간 : 2020-05-15 10:16:39.39
지역 : 세종
일일 확진자 수 : 0
누적 확진자 수 : 47

seq : 1744
수정 시간 : 2020-05-15 10:16:39.39
지역 : 울산
일일 확진자 수 : 0
누적 확진자 수 : 45

seq : 1743
수정 시간 : 2020-05-15 10:16:39.39
지역 : 대전
일일 확진자 수 : 0
누적 확진자 수 : 43

seq : 1742
수정 시간 : 2020-05-15 10:16:39.39
지역 : 광주
일일 확진자 수 : 0
누적 확진자 수 : 30

seq : 1741
수정 시간 : 2020-05-15 10:16:39.39
지역 : 인천
일일 확진자 수 : 5
누적 확진자 수 : 124

seq : 1740
수정 시간 : 2020-05-15 10:16:39.39
지역 : 대구
일일 확진자 수 : 3
누적 확진자 수 : 6868

seq : 1739
수정 시간 : 2020-05-15 10:16:39.39
지역 : 부산
일일 확진자 수 : 0
누적 확진자 수 : 144

seq : 1738
수정 시간 : 2020-05-15 10:16:39.39
지역 : 서울
일일 확진자 수

일일 확진자 수 : 0
누적 확진자 수 : 1361

seq : 1295
수정 시간 : 2020-04-21 10:20:02.02
지역 : 전남
일일 확진자 수 : 0
누적 확진자 수 : 15

seq : 1294
수정 시간 : 2020-04-21 10:20:02.02
지역 : 전북
일일 확진자 수 : 0
누적 확진자 수 : 17

seq : 1293
수정 시간 : 2020-04-21 10:20:02.02
지역 : 충남
일일 확진자 수 : 0
누적 확진자 수 : 141

seq : 1292
수정 시간 : 2020-04-21 10:20:02.02
지역 : 충북
일일 확진자 수 : 0
누적 확진자 수 : 45

seq : 1291
수정 시간 : 2020-04-21 10:20:02.02
지역 : 강원
일일 확진자 수 : 0
누적 확진자 수 : 53

seq : 1290
수정 시간 : 2020-04-21 10:20:02.02
지역 : 경기
일일 확진자 수 : 2
누적 확진자 수 : 658

seq : 1289
수정 시간 : 2020-04-21 10:20:02.02
지역 : 세종
일일 확진자 수 : 0
누적 확진자 수 : 46

seq : 1288
수정 시간 : 2020-04-21 10:20:02.02
지역 : 울산
일일 확진자 수 : 0
누적 확진자 수 : 43

seq : 1287
수정 시간 : 2020-04-21 10:20:02.02
지역 : 대전
일일 확진자 수 : 0
누적 확진자 수 : 39

seq : 1286
수정 시간 : 2020-04-21 10:20:02.02
지역 : 광주
일일 확진자 수 : 0
누적 확진자 수 : 30

seq : 1285
수정 시간 : 2020-04-21 10:20:02.02
지역 : 인천
일일 확진자 수 : 0
누적 확진자 수 : 92

seq : 1284
수정 시간 : 2020-04-21 10:20:02.02
지역 : 대구
일일 확진자 수 : 2
누적 확진자 수 : 6835

seq : 1283
수정 시간 : 2020-04-21 

seq : 499
수정 시간 : 2020-03-29 10:48:02.02
지역 : 경남
일일 확진자 수 : 3
누적 확진자 수 : 94

seq : 498
수정 시간 : 2020-03-29 10:48:02.02
지역 : 경북
일일 확진자 수 : 2
누적 확진자 수 : 1287

seq : 497
수정 시간 : 2020-03-29 10:48:02.02
지역 : 전남
일일 확진자 수 : 1
누적 확진자 수 : 9

seq : 496
수정 시간 : 2020-03-29 10:48:02.02
지역 : 전북
일일 확진자 수 : 2
누적 확진자 수 : 12

seq : 495
수정 시간 : 2020-03-29 10:48:02.02
지역 : 충남
일일 확진자 수 : 1
누적 확진자 수 : 127

seq : 494
수정 시간 : 2020-03-29 10:48:02.02
지역 : 충북
일일 확진자 수 : 0
누적 확진자 수 : 41

seq : 493
수정 시간 : 2020-03-29 10:48:02.02
지역 : 강원
일일 확진자 수 : 2
누적 확진자 수 : 34

seq : 492
수정 시간 : 2020-03-29 10:48:02.02
지역 : 경기
일일 확진자 수 : 15
누적 확진자 수 : 448

seq : 491
수정 시간 : 2020-03-29 10:48:02.02
지역 : 세종
일일 확진자 수 : 2
누적 확진자 수 : 46

seq : 490
수정 시간 : 2020-03-29 10:48:02.02
지역 : 울산
일일 확진자 수 : 0
누적 확진자 수 : 39

seq : 489
수정 시간 : 2020-03-29 10:48:02.02
지역 : 대전
일일 확진자 수 : 3
누적 확진자 수 : 34

seq : 488
수정 시간 : 2020-03-29 10:48:02.02
지역 : 광주
일일 확진자 수 : 0
누적 확진자 수 : 20

seq : 487
수정 시간 : 2020-03-29 10:48:02.02
지역 : 인천
일일 확진자 수 : 7
누적 확진자 수 :

지역 : 세종
일일 확진자 수 : 3
누적 확진자 수 : 6

seq : 117
수정 시간 : 2020-03-09 12:54:29.29
지역 : 울산
일일 확진자 수 : 0
누적 확진자 수 : 24

seq : 116
수정 시간 : 2020-03-09 12:54:29.29
지역 : 대전
일일 확진자 수 : 1
누적 확진자 수 : 19

seq : 115
수정 시간 : 2020-03-09 12:54:29.29
지역 : 광주
일일 확진자 수 : 2
누적 확진자 수 : 15

seq : 114
수정 시간 : 2020-03-09 12:54:29.29
지역 : 인천
일일 확진자 수 : 0
누적 확진자 수 : 9

seq : 113
수정 시간 : 2020-03-09 12:54:29.29
지역 : 대구
일일 확진자 수 : 190
누적 확진자 수 : 5571

seq : 112
수정 시간 : 2020-03-09 12:54:29.29
지역 : 부산
일일 확진자 수 : -1
누적 확진자 수 : 96

seq : 111
수정 시간 : 2020-03-09 12:54:29.29
지역 : 서울
일일 확진자 수 : 10
누적 확진자 수 : 130

seq : 110
수정 시간 : 2020-03-09 12:54:29.29
지역 : 합계
일일 확진자 수 : 248
누적 확진자 수 : 7382

seq : 109
수정 시간 : 2020-03-08 14:56:02.02
지역 : 제주
일일 확진자 수 : 0
누적 확진자 수 : 4

seq : 108
수정 시간 : 2020-03-08 14:56:02.02
지역 : 경남
일일 확진자 수 : 1
누적 확진자 수 : 83

seq : 107
수정 시간 : 2020-03-08 14:56:02.02
지역 : 경북
일일 확진자 수 : 32
누적 확진자 수 : 1081

seq : 106
수정 시간 : 2020-03-08 14:56:02.02
지역 : 전남
일일 확진자 수 : 0
누적 확진자 수 : 4

seq : 105
수정 시간 : 2020-03-08 1

In [139]:
# 공공데이터활용지원센터_코로나19 예방접종센터 현황
# https://www.data.go.kr/tcs/dss/selectFileDataDetailView.do?publicDataPk=15077603
# XML 파일의 태그들이 불편하게 되어있고 태그 인식 오류가 발생하여 bs 대신 et를 사용함

import requests
import xml.etree.ElementTree as et
import mysql.connector

url = 'https://api.odcloud.kr/api/15077586/v1/centers' # 서비스 URL
apiKey = 'serviceKey=' + 'WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D' # 인증키
page = 'page=' + str(1) # 페이지 번호
perPage = 'perPage=' + str(500) # 한 페이지 결과 수
returnType = 'returnType=' + 'XML' # JSON, XML 사용 가능, 대소문자 구분 해야 함
params = page + '&' + perPage + '&' + returnType # 파라메터 합치는 과정
open_url = url + '?' + apiKey + '&' + params # 요청 URL 완성

print(open_url + '\n') # 요청한 링크, 링크 누르면 데이터 확인 가능

res = requests.get(open_url) # 응답받은 데이터를 res에 저장
tree = et.fromstring(res.content) # et에 알맞게 변환

# col 태그 안의 내용들이 순서가 일정하지 않기 때문에 dictionary로 받아온 후 list에 재정렬
dictionaryList = [] # item 내부에 있는 dictionary를 넣을 list 생성

for item in tree.iter('item'): # 데이터에서 item 태그들을 찾고
    dic = {} # dictionary 생성
    for col in item.iter('col'): # item 태그안의 col 태그를 찾아 하나씩 반복
        dic[col.get('name')] = col.text # 찾은 정보를 dictionary에 key와 value를 알맞게 추가
        #print(col.get('name') ,":", col.text)
    #print(dic)
    dictionaryList.append(dic) # list에 추가
    #print()

result = [] # 쿼리를 편하게 사용하기 위해 위의 list 데이터를 정리하기 위한 list 생성
for item in dictionaryList:
    # list안의 dictionary 데이터를 쿼리에 알맞게 정렬
    data = (item['centerName'], item['facilityName'], item['phoneNumber'], item['sido'], item['address'], item['zipCode'], item['lat'], item['lng'])
    print(data)
    result.append(data) # list에 추가
    
# MariaDB 커넥션 생성
'''
try:
    conn = mysql.connector.connect(
        user="multi",
        password="multi",
        host="melonbottle.synology.me",
        port=3308,
        database="multi"
    )
    print(conn)
except mysql.connector.Error as e:
    print(e)
'''

cur = conn.cursor() # 커서 생성

sql = "INSERT IGNORE INTO covid_vaccine_center VALUES(%s, %s, %s, %s, %s, %s, %s, %s)" # 쿼리 생성

cur.executemany(sql, result) # 쿼리 실행

conn.commit() # 커밋 

# close 작업 실행
cur.close()
conn.close()

https://api.odcloud.kr/api/15077586/v1/centers?serviceKey=WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D&page=1&perPage=500&returnType=XML

('코로나19 중앙 예방접종센터', '국립중앙의료원 D동', '02-2260-7114', '서울특별시', '서울특별시 중구 을지로 39길 29', '4562', '37.567817', '127.004501')
('코로나19 중부권역 예방접종센터', '천안시 실내배드민턴장 1층', None, '충청남도', '충청남도 천안시 동남구 천안대로 357', '31212', '36.779887', '127.164717')
('코로나19 호남권역 예방접종센터', '조선대학교병원 의성관 5층', '062-220-3739', '광주광역시', '광주광역시 동구 필문대로 365', '61452', '35.139465', '126.925563')
('코로나19 영남권역 예방접종센터', '양산 부산대병원 기숙사동 1층', '055-360-6701', '경상남도', '경상남도 양산시 물금읍 금오로 20', '50612', '35.3239', '129.009337')
('코로나19 대구광역시 중구 예방접종센터', '계명대학교 대구동산병원 별관', '053-661-3955', '대구광역시', '대구시 중구 달성로 56', '41931', '35.869985', '128.583716')
('코로나19 서울특별시 성동구 예방접종센터', '성동구청 대강당(3층)', '02-2286-5084', '서울특별시', '서울특별시 성동구 고산자로 270', '4750', '37.563457', '127.036981')
('코로나19 부산광역시 부산진구 예방접종센터', '부산시민공원 시민사랑채', '051-605-8633', '부산광역시', '부산 부산진구 시민공원로

In [ ]:
# 행정안전부_재난문자방송 발령현황
# https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=3058822
# 실시간 확진자 수 파악에 사용 예정

import requests
from bs4 import BeautifulSoup
import mysql.connector
import datetime as dt
import re
import time as t

# 프로그램 시작 후 현재 시간을 갱신 시간으로 입력, 이후에 데이터 갱신 시간을 비교하여 원하는 데이터 입력
update_date = dt.datetime.now()
# update_date = dt.datetime(2021, 5, 18, 11, 0, 0) # 임의의 갱신 시간
print('Program Start! Time : ' + update_date.strftime('%Y-%m-%d %H:%M:%S'))
update_date = update_date - dt.timedelta(minutes=10) # 프로그램 시작 10분 전 데이터부터 집계 시작

# API에서 사용되는 지역명과 DB에서 사용되는 지역명을 일치 시키기 위한 list
location_list = [
    '서울특별시', '인천광역시', '광주광역시', '대전광역시', '대구광역시', '부산광역시', '울산광역시', '세종특별자치시', '경기도', '강원도',
    '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도'
]
db_location_list = [
    'seoul', 'incheon', 'gwangju', 'daejeon', 'daegu', 'busan', 'ulsan', 'sejong', 'gyeonggi', 'gangwon',
    'chungbuk', 'chungnam', 'jeonbuk', 'jeonnam', 'gyeongbuk', 'gyeongnam', 'jeju'
]

url = 'http://apis.data.go.kr/1741000/DisasterMsg3/getDisasterMsg1List' # 서비스 URL
apiKey = 'serviceKey=' + 'WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D' # 인증키
pageNo = 'pageNo=' + str(1) # 페이지 번호
numOfRows = 'numOfRows=' + str(30) # 한 페이지 결과 수
returnType = 'type=' + 'xml' # json, xml 사용 가능, type는 python 기본 함수라서 returnType으로 변수명 변경
params = pageNo + '&' + numOfRows + '&' + returnType # 파라메터 합치는 과정
open_url = url + '?' + apiKey + '&' + params # 요청 URL 완성

print(open_url + '\n') # 요청한 링크, 링크 누르면 데이터 확인 가능

# 1분 마다 while 반복, update_date 이후의 데이터를 입력.
# 입력 된 데이터가 있을 시 update_date 수정,
# 아닐 경우 update_date를 유지하고 코드 실행 반복
while(True):
    print('Start!')
    res = requests.get(open_url) # 응답받은 데이터를 res에 저장
    soup = BeautifulSoup(res.content, 'html.parser') # bs에 알맞게 변환

    items = soup.findAll('row') # 데이터에서 row 태그들을 찾아 ResultSet으로 반환

    result = [] # row 태그 안의 내용을 알맞게 넣을 list 생성

    for one in items:
        # 재난 문자 입력 시간들을 추출하기 위한 코드
        split_create_date = str(one.find('create_date').string).split(' ') # ex) 2021/05/18 19:01:01
        date = split_create_date[0] # ex) 2021/05/18
        time = split_create_date[1] # ex) 19:01:01
        split_date = date.split('/') 
        split_time = time.split(':')
        year = int(split_date[0]) # 년
        month = int(split_date[1]) # 월
        day = int(split_date[2]) # 일
        hour = int(split_time[0]) # 시
        minute = int(split_time[1]) # 분
        second = int(split_time[2]) # 초
        datetime = dt.datetime(year, month, day, hour, minute, second) # 추출한 시간들로 datetime 생성

        split_location = str(one.find('location_name').string).split(' ') # ex) 경기도 ㅇㅇ시
        location = split_location[0] # ex) 경기도

        msg = str(one.find('msg').string) # 재난 문자 내용
        
        # 출력 태그들
        #print('문자 발송 일시 : ' + datetime.strftime('%Y-%m-%d %H:%M:%S'))
        #print('문자 발송 날짜 : ' + date)
        #print('문자 발송 시간 : ' + time)
        #print('도, 광역, 특별시 : ' + location)
        #print('메시지 내용 : ' + msg)   
        #print()
        
        # 재난 문자 내용 중 '확진자'라는 단어가 시작하는 위치와 '명'으로 끝나는 위치의 index 추출
        startIndex = msg.find('확진자')
        endIndex = msg.find('명')
        
        # 확진자 수를 저장할 int 데이터 초기화
        count = 0
        
        # 두 index가 제대로 추출 되었으면
        if startIndex != -1 and endIndex != -1 and startIndex < endIndex:
            numMsg = msg[startIndex : endIndex + 1] # ex) 확진자 18명
            #print(numMsg)
            num = re.findall("\d+", numMsg) # numMsg 문자열에서 숫자만 추출 
            if(len(num) == 1): # 추출한 값 중 앞의 값을 우선으로
                count = int(num[0]) # 확진자 수 입력
                #print(count)
        #print()    
        
        # 확진자 수가 0명 초과하고 갱신되지 않은 데이터라면 
        if count > 0 and datetime > update_date:
            # 위에서 추출한 location 정보를 DB에 맞는 location 값으로 변경
            location = db_location_list[location_list.index(location)]
            data = [count, date, location] # 필요한 데이터를 list에 저장
            #print(data)
            result.append(data) # list에 추가
    
    # 삽입 될 값들 출력
    print('Result : ')
    print(result)
    
    # MariaDB 커넥션 생성
    '''
    try:
        conn = mysql.connector.connect(
            user="multi",
            password="multi",
            host="melonbottle.synology.me",
            port=3308,
            database="multi"
        )
        #print(conn)
    except mysql.connector.Error as e:
        print(e)
    '''
    
    cur = conn.cursor() # 커서 생성
    now = dt.datetime.now() # 현재 시간 받아와서
    now_str = update_date.strftime('%Y-%m-%d') # 연월일 str 데이터로 변경
    
    # covid_live table에 현재 날짜 데이터가 있는지 확인
    sql = "SELECT * FROM covid_live WHERE live_date = '" + now_str + "'" # 쿼리 생성
    cur.execute(sql) # 쿼리 실행
    sql_result = cur.fetchone() # 결과 값 입력

    # 결과값이 없다면 = 현재 날짜 데이터가 없음
    if sql_result is None: 
        print('Insert New Date! Time : ' + now_str)
        # 현재 날짜 데이터를 입력 할 쿼리 생성
        sql = "INSERT INTO covid_live VALUES ( '"+ now_str +"', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)" 
        cur.execute(sql) # 쿼리 실행
    
    for one in result:
        # 결과값을 입력할 쿼리 생성
        sql = "UPDATE covid_live SET " + one[2] + " = " + one[2] + " + %s WHERE live_date = %s"
        del one[2] # list에서 필요 없어진 데이터를 삭제
        cur.execute(sql, one) # 쿼리 실행
    
    conn.commit() # 커밋
    
    # close 문구 실행
    cur.close()
    conn.close()
    
    # 결과값이 있다면 = 데이터가 입력 되었으면
    if result:
        print('Updated Datetime!')
        update_date = dt.datetime.now() # 최근 데이터 갱신 시간을 현재 시간으로 변경
    else:
        now = dt.datetime.now() # 코드 실행 시간 출력
        print('No Update! Time : ' + now.strftime('%Y-%m-%d %H:%M:%S'))
        
    print('Recent Update Time : ' + update_date.strftime('%Y-%m-%d %H:%M:%S'))
    print('Sleep 60 Sec...')
    print()
    t.sleep(60) # 60초 뒤 코드 재실행

Program Start Time : 2021-05-18 18:15:27
http://apis.data.go.kr/1741000/DisasterMsg3/getDisasterMsg1List?serviceKey=WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D&pageNo=1&numOfRows=30&type=xml

Start!
[3, '2021/05/18', 'gyeonggi']
Result : 
[[3, '2021/05/18', 'gyeonggi']]
Insert New Date! Time : 2021-05-18
Updated Datetime!
Recent Update Time : 2021-05-18 18:15:27
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 18:16:27
Recent Update Time : 2021-05-18 18:15:27
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 18:17:28
Recent Update Time : 2021-05-18 18:15:27
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 18:18:29
Recent Update Time : 2021-05-18 18:15:27
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 18:19:29
Recent Update Time : 2021-05-18 18:15:27
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 18:20:30
Recent Update Time : 2021-05-18 18:15:27


Start!
Result : 
[]
No Update! Time : 2021-05-18 19:48:29
Recent Update Time : 2021-05-18 18:51:26
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 19:49:29
Recent Update Time : 2021-05-18 18:51:26
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 19:50:30
Recent Update Time : 2021-05-18 18:51:26
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 19:51:30
Recent Update Time : 2021-05-18 18:51:26
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 19:52:31
Recent Update Time : 2021-05-18 18:51:26
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 19:53:31
Recent Update Time : 2021-05-18 18:51:26
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 19:54:32
Recent Update Time : 2021-05-18 18:51:26
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 19:55:32
Recent Update Time : 2021-05-18 18:51:26
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-05-18 19:56:33
Recent Update 